In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
import cv2, os, sys
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from scipy import spatial 
from operator import itemgetter

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)

In [ ]:
# Divide datas into train and test
imgs_path = "../input/shark-species/sharks/"

X_train, y_train, X_test, y_test = [], [], [], []

for dirname in os.listdir(imgs_path):
    dir_name = os.path.join(imgs_path, dirname)
    dir_content = os.listdir(dir_name)
    for filename in dir_content:
        if filename is not None:
            #print(filename)
            img_path = os.path.join(dir_name, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (75,75))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                label = filename.split('_')[0]

                if np.random.rand() < 0.5:
                    X_train.append(img)
                    y_train.append(label)
                else:
                    X_test.append(img)
                    y_test.append(label)

In [ ]:
# Display our sharks data shape
X_train = np.array(X_train)
X_test = np.array(X_test)
print(X_train.shape, X_test.shape)

In [ ]:
# Normalize our sharks images
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
base_model = tf.keras.applications.InceptionV3(
    weights='imagenet',
    input_shape=(75, 75, 3),
    include_top=False)

In [ ]:
new_X_test = []
for x in X_test:
    new_X_test.append(cv2.resize(x, (75,75)))
new_X_test = np.array(new_X_test)
new_X_test.shape

In [ ]:
pred = base_model.predict(new_X_test)

In [ ]:
index = 1
plt.imshow(new_X_test[index].reshape(75,75,3))
plt.gray()
plt.show()

In [ ]:
img_to_find = pred[index]

In [ ]:
# Get similarities with cosinus
def custom_cosine_sim(a,b):
    return np.dot(a, b) / ( nalgnp.linalg.norm(a) * np.li.norm(b))

In [ ]:
# Calculate similarities between X_test and decoded images
cosine_list = []
for index_image,xt in enumerate(pred):
    result = 1 - spatial.distance.cosine(img_to_find.reshape(-1), xt.reshape(-1))
    cosine_list.append(dict({'res':result, 'i':index_image}))
    
cosine_list.sort(key=itemgetter('res'), reverse=True)

fig, ax = plt.subplots(nrows=1, ncols=10,figsize=(20, 4))
plt.gray()
for indice, row in enumerate(ax):
    print (cosine_list[indice]['i'])
    row.imshow(new_X_test[cosine_list[indice]['i']].reshape(75,75,3))

plt.show()